In [0]:
!pip install -qU openai
!pip install -qU langchain
!pip install -qU langchain-openai
#!pip install -qU langchain-experimental
!pip install -q langchain-experimental==0.0.49
!pip install -qU langchain-community
!pip install -qU langchainhub 
!pip install -q numexpr
#!pip install -q tavily-python==0.3.0 
#!pip install -qU tiktoken
!pip install wikipedia==1.4.0
!pip install -q duckduckgo-search

dbutils.library.restartPython()

# LangChain: Agents  
Sources: [1](https://brightinventions.pl/blog/introducing-langchain-agents-tutorial-with-example/), [2](https://levelup.gitconnected.com/using-langchain-csv-agent-for-performing-analytical-tasks-79d073fcbde7), [3](https://python.langchain.com/docs/integrations/retrievers/tavily)  

#### The core idea of agents is to use an LLM to choose a sequence of actions to take. 
+ In chains, a sequence of actions is hardcoded (in code).  
+ In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

![Agents vs Chains](https://brightinventions.pl/static/98578848ff94ebbf2fd58883fbe3e780/d9199/llm_agents_loop.png)  

Agents are crucial for handling tasks ranging from simple automated responses to complex, context-aware interactions.
For example, you may have an agent integrated with Google Search, Wikipedia and OpenAI LLM. With given agent tools, they can search for results in Google, then use retrieved context in Wikipedia tool to find detailed information and expand the context. Bear in mind that you have to put clearly defined instructions to make sure that the agent will invoke tools in a proper order.

![Architecture](https://brightinventions.pl/static/5acdb7bfc6e66980738c29a5e9c96d76/d9199/llm_agents.png)


Examples of Agents:
1. Web search tool: You can easily add different types of web search as an available action to your agent. It might be Google Search, Tavily Search, DuckDuckGo and many others.
2. Embedding search in vector database: You can create a tool from retriever and describe it as you need, so the agent will use this tool to get some kind of data doing e.g. similarity check and embedding model.
3. Doing some actions: Your agent can be multipurpose one. For example, it might be searching for some kind of information on the internet, doing the reasoning step, and after all invoking action to create a Jira issue.
4. API integration tool: There are many API integration done for the LangChain framework, all you need to do is take the API key, install the package and attach the tool to the agent.
5. Custom tool: You can write your own tool, refer to the documentation to see how to do it. It might be integration with your internal API, your document system, and many others!

LangChain Agent types: LangChain categorizes agents based on several dimensions:  
+ model type;
+ support for chat history;
+ multi-input tools;
+ parallel function calling;
+ required model parameters.

It is important to choose the option that fits to your use case:  
1. OpenAI functions: There are certain models fine-tuned where input is a bit different than usual. There are special functions that can be called and the role of this agent is to determine when it should be invoked. This agent is designed to work with this kind of OpenAI model. It supports chat history.  
2. OpenAI tools: This agent is designed to work with OpenAI tools, so its role is to interact and determine whether to use e.g. image generation tool or another built-in one. The main difference between OpenAI function is the fact that the function is trying to find the best fitting algorithm/part of an algorithm to do better reasoning, while OpenAI tool is about built-in tools like image generation, and executing code. It supports chat history.  
3. XML Agent: There are models, where reasoning/writing XML is on a very advanced level (a good example is Anthropic Claude's model). If you're operating on XML files, that might be the right one to be considered. It supports chat history.  
4. JSON Chat Agent: Several LLMs available in the market are particularly handy when it comes to reading JSON. JSON is also a very common standard of some e.g. entity representation. If you're building some kind of integration that operates on JSON files and the model is supporting it, you can try to use this agent. It supports chat history.  
5. Structured chat: Intended for multi-input tools. It supports chat history.  
6. ReAct agent: Made for simple models (LLM - not conversational). It supports chat history.  
7. Self-ask with search: This kind of agent supports only one tool as an input. The main goal is to divide your query into smaller ones, use tools to get the answer, and then combine it into a full answer to your question. This kind of agent doesn’t support chat history.  


Example Notebooks:
+ https://github.com/langchain-ai/langchain/blob/master/cookbook/two_agent_debate_tools.ipynb  
+ https://github.com/langchain-ai/langchain/blob/master/cookbook/databricks_sql_db.ipynb  
+ https://github.com/langchain-ai/langchain/blob/master/cookbook/baby_agi.ipynb  
+ https://github.com/langchain-ai/langchain/blob/master/cookbook/baby_agi_with_agent.ipynb  



In [0]:
import os
import glob
from pathlib import Path
from IPython.display import display, Markdown
import pandas as pd
import tiktoken
#from funcy import lcat, lmap, linvoke
import warnings
warnings.filterwarnings('ignore')

## Langchain LLM Objects
import openai
import langchain
from langchain.llms import OpenAI
from langchain_openai import AzureOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

## Langchain Chains 
#from langchain.chains import ConversationChain
#from langchain.chains import LLMChain
#from langchain.chains import ConversationChain
#from langchain.chains import ConversationalRetrievalChain
#from langchain.chains import RetrievalQA
#from langchain.chains.mapreduce import MapReduceChain
#from langchain.chains.summarize import load_summarize_chain
#from langchain.chains.question_answering import load_qa_chain
#from langchain.chains import SimpleSequentialChain
#from langchain.chains import SequentialChain
#from langchain.chains.router import MultiPromptChain
#from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
#from langchain.chains import PALChain
#from libs.experimental.langchain.experimental.pal_chain.base import PALChain
from langchain.chains import LLMMathChain

## Langchain Schemas 
from langchain.schema import HumanMessage

## Langchain Agents  
from langchain import hub
from langchain.agents import tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain.python import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.agents.agent_toolkits import create_spark_dataframe_agent

#from langchain.tools.tavily_search import TavilySearchResults
#from langchain.utilities.tavily_search import TavilySearchAPIWrapper

openai.api_type = "azure"
azure_endpoint = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#azure_endpoint = "https://chatgpt-summarization.openai.azure.com/"

openai.api_version = "2023-07-01-preview"
openai.api_key = os.environ["OPENAI_API_KEY"]

deployment_name = "model-gpt-35-turbo"
openai_model_name = "gpt-35-turbo"

client = AzureOpenAI(api_key=openai.api_key,
                     api_version=openai.api_version,
                     azure_endpoint=azure_endpoint,
                     )

chat = AzureChatOpenAI(azure_endpoint=azure_endpoint,
                       openai_api_version=openai.api_version,
                       deployment_name=deployment_name,
                       openai_api_key=os.environ["OPENAI_API_KEY"],
                       openai_api_type=openai.api_type,
                       )


## Simple Agents  

##### Most of the time, the LLM gives a satisfactory answer to what we need:  

In [0]:
query = "What is the sixt element in the fibonacci sequence"
response = chat(messages=[HumanMessage(content=query)])
print(response.content)
print(response)

##### But in this second case, only the Python agent is able to figure out how to provide the missing information (the year)

In [0]:
query = "We are in 2024. In which day of week the Austrian National Holiday will be next year?"
response = chat(messages=[HumanMessage(content=query)])
print(response.content)
print(response)

In [0]:
agent = create_python_agent(chat, tool=PythonREPLTool(), verbose=True, handle_parsing_errors=True)
#langchain.debug=True
agent.run(query) 
#langchain.debug=False

#### We can use a Python Agent for different tasks:

##### Task #1 

In [0]:
customer_list = [["Renato", "Fiacchini"], 
                 ["Charles", "Aznavour"],
                 ["Bernhard", "Thomas"],
                 ["Joel", "Billy"], 
                 ["David","Goliah"], 
                 ["Brani","Acer"],
                 ["Georg","Curious"]
                ]

In [0]:
#langchain.debug=True
agent.run(f"""Sort these customers by last name and then first name and print the output: {customer_list}""") 
#langchain.debug=False

##### Task #2

In [0]:
agent.run(f"""What are the char codes for each name on the list in hexadecimal: {customer_list}""")

##### We have to be aware that the models are incorporating themselves math functionalities.  
See the example:

In [0]:
def collatz(n):
    print(n)
    while n != 1:
        if n%2 == 0:
            n /= 2
        else:
            n = (3*n) + 1
        print(int(n), end=" ")

collatz(45)

In [0]:
query1 = "Print the Collatz sequence for the number 45"
query2 = "What is the biggest prime number below 1 billion?"
response = chat(messages=[HumanMessage(content=query1)])
print(response.content)

In [0]:
langchain.debug=True
print(agent.run(query1))

## Loading specific Agents

In [0]:
tools = load_tools(["llm-math","wikipedia"], llm=chat)

In [0]:
agent= initialize_agent(tools,
                        chat,
                        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                        handle_parsing_errors=True,
                        verbose = True)

Mind the message ponting out to the new architecture:  
LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.  

#### 1 - Wikipedia (blocked by Firewall)

In [0]:
#langchain.debug=True
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)
#langchain.debug=False

#### 2 - LLM-Math

In [0]:
langchain.debug=False
agent("How much is 25% of 300?")

#### 3 - Duck Duck Go (blocked by Firewall)  

In [0]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [0]:
search.run("Obama's first name?")

#### 4 - Pandas Agent  

In [0]:
df = pd.read_csv('../../Data/Data.csv')
df.head(10)

In [0]:
langchain.debug=False

pandas_agent = create_pandas_dataframe_agent(llm=chat, 
                                             df=df, 
                                             verbose=True, 
                                             agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,)

In [0]:
pandas_agent.run("how many rows are there?")

In [0]:
pandas_agent.run("Is there a review about L'Or? Please show me it")

## Define your own tool

In [0]:
from datetime import date

In [0]:
@tool
def my_color(text: str) -> str:
    """I use different colors of T-shirts
    depending on the day of month.
    I use White in the even days and
    I use Blue in the odd days
    This function tells me which color 
    to use"""
    day = date.today().day
    if day%2 == 0:
        return "White"
    return "Blue"

In [0]:
agent= initialize_agent([my_color],
                        chat,
                        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                        handle_parsing_errors=True,
                        verbose = False)

In [0]:
langchain.debug=False

try:
    result = agent("What t-shirt color should I use today")
    print(result)
except: 
    print("exception on external access")

langchain.debug=False